In [1]:
%matplotlib qt
import matplotlib.pyplot as mpl
import numpy as np
import cnmodel.util.sound as SND
from pylibrary.plotting import plothelpers as PH
def reset_style():
#     sns.set_style(rc={"pdf.fonttype": 42})
    mpl.style.use("~/.matplotlib/figures.mplstyle")
mpl.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})
# for Palatino and other serif fonts use:
# mpl.rcParams.update({
#     "text.usetex": True,
#     "font.family": "serif",
#     "font.serif": ["Palatino"],
# })

Make a tone pip
---------------

In [2]:
S = SND.TonePip(rate=1e5, duration=0.3, f0=250., dbspl=90., pip_duration=0.2, pip_start=[0.05], ramp_duration=0.0025)
t = S.generate()
f, ax = mpl.subplots(1,1)
PH.cleanAxes(ax)
PH.noaxes(ax)
ax.plot(S.time, t)
mpl.show()

Make a short AN spike train from the tone pip
---------------------------------------------

In [3]:
from neuron import h
from cnmodel.protocols import Protocol
from cnmodel import cells
from cnmodel.util import custom_init

preCell = cells.DummySGC(cf=4000, sr=2)
postCell = cells.Bushy.create()
synapse = preCell.connect(postCell)
seed = 9
temp=34.
dt=0.025
simulator = 'cochlea'
stim = SND.TonePip(rate=100e3, duration=0.1, f0=4000, dbspl=80,
                          ramp_duration=2.5e-3, pip_duration=0.04, 
                          pip_start=[0.02])
preCell.set_sound_stim(stim, seed=seed, simulator=simulator)
print(dir(preCell))
# vm = postCell.soma(0.5)._ref_v
# im = postCell.soma(0.5)._ref_i
xmtr = [None]*30
for i in range(30):
    xmtr[i] = synapse.terminal.relsite._ref_XMTR[i]
    synapse.terminal.relsite.Dep_Flag = False


VClamp = h.SEClamp(0.5, sec=postCell.soma)
VClamp.dur1 = 500.
VClamp.amp1 = -65.
VClamp.dur2 = 500.
VClamp.amp2 = -65.
VClamp.dur3 = 500.
VClamp.amp3 = -65.


VClamp.vc = -65.
VClamp.rs = 1e-9
vm = h.Vector()
vm.record(postCell.soma(0.5)._ref_v, sec=postCell.soma)
im = h.Vector()
im.record(VClamp._ref_i, sec=postCell.soma)
t = h.Vector()
t.record(h._ref_t)
sgc = h.Vector()
sgc.record(preCell.soma(0.5)._ref_v, sec=preCell.soma)

h.tstop = 100.0 # duration of a run
h.celsius = temp
h.dt = dt
v_init = -65

custom_init()
h.run()
print(preCell._spiketrain)
f, ax = mpl.subplots(3,1)
ax[0].plot(stim.time*1e3, stim.sound)
pst = preCell._spiketrain
opst = np.ones_like(pst)
ax[1].plot(np.vstack((pst, pst, [np.nan]*len(pst))),
           np.vstack((opst*0.9, opst*1.1, [np.nan]*len(pst))),
                
          color='k')

ax[2].plot(t, im)
for i in range(3):
    ax[i].set_xlim(0, 100.)
ax[1].set_ylim(0.5, 1.5)
mpl.show()


How to cite *cochlea*:

Rudnicki M. and Hemmert W. (2009—). Cochlea: inner ear models in
Python.  https://github.com/mrkrd/cochlea/

@Misc{Rudnicki2009-Cochlea,
  author       = {Rudnicki, M. and Hemmert, W.},
  title        = {Cochlea: inner ear models in Python},
  year         = {2009--},
  howpublished = {https://github.com/mrkrd/cochlea},
  url          = {https://github.com/mrkrd/cochlea},
}



['R_a', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cf', '_ghk', '_lambda_f', '_simulator', '_sound_stim', '_spiketrain', '_sr', '_stvec', 'add_axon', 'add_section', 'adjust_na_chans', 'all_sections', 'areaMap', 'area_method', 'axnode', 'axon', 'axonsf', 'c_m', 'cell_initialize', 'celltype', 'cf', 'channel_manager', 'check_all_mechs', 'check_temperature', 'computeAreas', 'compute_rmrintau', 'connect', 'create', 'decorate', 'dendrite', 'distances', 'do_morphology', 'e_h', 'e_k', 'e_leak', 'e_na', 'find_i0', 'from_section', 'g_convert', 'generate_spiketrain', 'get_all_mechs', 'get_cellpars', 'get_initial_pars', 'get_mechs', 'get_post_sec', 'get_section_type', 'hr', 'i_currents', 'i_test_rang